In [1]:
import pandas as pd
import numpy as np

In [5]:
y = pd.read_pickle('ytrain.pkl')

In [6]:
y

,target
0,0
1,1
2,2
3,3
4,4
...,...
84911,18
84912,12
84913,1
84914,7


In [10]:
X = pd.read_pickle('x_train_new.pkl')

In [11]:
X.head()

,designation,description,productid,imageid,text
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,<NA>,3804725264,1263597046,Olivia: Personalisiertes Notizbuch / 150 Seite...
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,<NA>,436067568,1008141237,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,<NA>,50418756,457047496,Peluche Donald - Europe Disneyland 2000 (Mario...
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,La Guerre Des TuquesLuc a des id&eacute;es de ...


In [32]:
#!pip install swifter

In [14]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import swifter

stop_words = stopwords.words('french') + stopwords.words('english') + stopwords.words('german') +['plus', 'peut', 'tout', 'etre', 'sans', 'dont', 'aussi', 'comme', 'meme', 'bien','leurs', 'elles', 'cette','celui', 'ainsi', 'encore', 'alors', 'toujours', 'toute','deux', 'nouveau', 'peu', 'car', 'autre', 'jusqu', 'quand', 'ici', 'ceux', 'enfin', 'jamais', 'autant', 'tant', 'avoir', 'moin', 'celle', 'tous', 'contre', 'pourtant', 'quelque', 'toutes', 'surtout', 'cet', 'comment', 'rien', 'avant', 'doit', 'autre', 'depuis', 'moins', 'tre', 'souvent', 'etait', 'pouvoir', 'apre', 'non', 'ver', 'quel', 'pourquoi', 'certain', 'fait', 'faire', 'sou', 'donc', 'trop', 'quelques', 'parfois', 'tres', 'donc', 'dire', 'eacute', 'egrave', 'rsquo', 'agrave', 'ecirc', 'nbsp', 'acirc', 'apres', 'autres', 'ocirc', 'entre', 'sous', 'quelle']

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

X.text = X.text.swifter.apply(lambda x :preprocess_sentence(x))
X.head()

Pandas Apply:   0%|          | 0/84916 [00:00<?, ?it/s]

,designation,description,productid,imageid,text
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,<NA>,3804725264,1263597046,olivia personalisiertes notizbuch seiten punkt...
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,<NA>,436067568,1008141237,journal arts art marche salon art asiatique pa...
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,grand stylet ergonomique bleu gamepad nintendo...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,<NA>,50418756,457047496,peluche donald europe disneyland marionnette d...
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,guerre tuquesluc grandeur . veut organiser jeu...


In [15]:
from sklearn.model_selection import train_test_split
X_text_train, X_text_test, y_train, y_test = train_test_split(X.text, y, test_size=0.2, random_state=1234)

In [31]:
#!pip install tensorflow

In [18]:
import tensorflow as tf
# Définition du tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
# Mettre à jour le dictionnaire du tokenizer
tokenizer.fit_on_texts(X_text_train)

In [19]:
# Définition des dictionnaires
word2idx = tokenizer.word_index
idx2word = tokenizer.index_word
vocab_size = tokenizer.num_words

In [20]:
X_train = tokenizer.texts_to_sequences(X_text_train)

In [21]:
X_test = tokenizer.texts_to_sequences(X_text_test)

In [22]:
maxlen = 500
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')

In [23]:
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

# Modèle : RNN GRUCELL

In [28]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, RNN, GRUCell, Dropout
embedding_dim = 200

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(RNN(GRUCell(128), return_sequences=True))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(27, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 200)         2000000   
                                                                 
 rnn_2 (RNN)                 (None, None, 128)         126720    
                                                                 
 dropout_4 (Dropout)         (None, None, 128)         0         
                                                                 
 global_average_pooling1d_2   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 256)               33024     
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                      

In [29]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train.values,
    batch_size = 64,
    epochs=10,
    validation_data = [X_test, y_test.values])

Epoch 1/10
1062/1062 [==============================] - 1377s 1s/step - loss: 1.8048 - accuracy: 0.4549 - val_loss: 1.0505 - val_accuracy: 0.6843
Epoch 2/10
1062/1062 [==============================] - 947s 892ms/step - loss: 0.8713 - accuracy: 0.7405 - val_loss: 0.8307 - val_accuracy: 0.7476
Epoch 3/10
1062/1062 [==============================] - 837s 788ms/step - loss: 0.6567 - accuracy: 0.8033 - val_loss: 0.8038 - val_accuracy: 0.7639
Epoch 4/10
1062/1062 [==============================] - 796s 749ms/step - loss: 0.5378 - accuracy: 0.8355 - val_loss: 0.8149 - val_accuracy: 0.7684
Epoch 5/10
1062/1062 [==============================] - 810s 763ms/step - loss: 0.4552 - accuracy: 0.8600 - val_loss: 0.8216 - val_accuracy: 0.7695
Epoch 6/10
1062/1062 [==============================] - 817s 769ms/step - loss: 0.3844 - accuracy: 0.8789 - val_loss: 0.9217 - val_accuracy: 0.7668
Epoch 7/10
1062/1062 [==============================] - 799s 752ms/step - loss: 0.3309 - accuracy: 0.8960 - val_lo

In [30]:
model.evaluate(X_test, y_test.values, batch_size=128)

133/133 [==============================] - 33s 245ms/step - loss: 1.2312 - accuracy: 0.7696


[1.231240153312683, 0.7696067094802856]

# Modèle : RNN LSTM CELL

In [34]:
from tensorflow.keras.layers import LSTMCell

In [35]:
embedding_dim = 200

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(RNN(LSTMCell(128), return_sequences=True))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(27, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 200)         2000000   
                                                                 
 rnn_3 (RNN)                 (None, None, 128)         168448    
                                                                 
 dropout_6 (Dropout)         (None, None, 128)         0         
                                                                 
 global_average_pooling1d_3   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                      

In [36]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train.values,
    batch_size = 64,
    epochs=10,
    validation_data = [X_test, y_test.values])

Epoch 1/10
1062/1062 [==============================] - 803s 755ms/step - loss: 2.6560 - accuracy: 0.2028 - val_loss: 2.1729 - val_accuracy: 0.2916
Epoch 2/10
1062/1062 [==============================] - 939s 884ms/step - loss: 1.9471 - accuracy: 0.3648 - val_loss: 1.6845 - val_accuracy: 0.4548
Epoch 3/10
1062/1062 [==============================] - 1042s 981ms/step - loss: 1.3541 - accuracy: 0.5614 - val_loss: 1.2129 - val_accuracy: 0.6236
Epoch 4/10
1062/1062 [==============================] - 1256s 1s/step - loss: 0.9921 - accuracy: 0.6914 - val_loss: 0.9991 - val_accuracy: 0.7083
Epoch 5/10
1062/1062 [==============================] - 1022s 962ms/step - loss: 0.7839 - accuracy: 0.7672 - val_loss: 0.9327 - val_accuracy: 0.7358
Epoch 6/10
1062/1062 [==============================] - 1012s 953ms/step - loss: 0.6590 - accuracy: 0.8048 - val_loss: 0.9338 - val_accuracy: 0.7412
Epoch 7/10
1062/1062 [==============================] - 1034s 974ms/step - loss: 0.5692 - accuracy: 0.8318 - va

In [37]:
model.evaluate(X_test, y_test.values, batch_size=128)

133/133 [==============================] - 43s 320ms/step - loss: 1.0030 - accuracy: 0.7531


[1.002968192100525, 0.7531206011772156]

# Modèle : RNN (SIMPLE RNN CELL)

In [38]:
from tensorflow.keras.layers import SimpleRNNCell

In [39]:
embedding_dim = 200

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(RNN(SimpleRNNCell(128), return_sequences=True))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(27, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 200)         2000000   
                                                                 
 rnn_4 (RNN)                 (None, None, 128)         42112     
                                                                 
 dropout_8 (Dropout)         (None, None, 128)         0         
                                                                 
 global_average_pooling1d_4   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_8 (Dense)             (None, 256)               33024     
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                      

In [40]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train.values,
    batch_size = 64,
    epochs=10,
    validation_data = [X_test, y_test.values])

Epoch 1/10
1062/1062 [==============================] - 313s 294ms/step - loss: 2.8420 - accuracy: 0.1679 - val_loss: 2.4863 - val_accuracy: 0.1949
Epoch 2/10
1062/1062 [==============================] - 336s 317ms/step - loss: 2.2756 - accuracy: 0.2876 - val_loss: 2.5540 - val_accuracy: 0.2209
Epoch 3/10
1062/1062 [==============================] - 334s 315ms/step - loss: 1.9782 - accuracy: 0.3667 - val_loss: 1.7750 - val_accuracy: 0.4189
Epoch 4/10
1062/1062 [==============================] - 327s 308ms/step - loss: 1.7092 - accuracy: 0.4442 - val_loss: 1.6238 - val_accuracy: 0.4832
Epoch 5/10
1062/1062 [==============================] - 323s 304ms/step - loss: 1.6295 - accuracy: 0.4805 - val_loss: 1.6560 - val_accuracy: 0.4813
Epoch 6/10
1062/1062 [==============================] - 344s 324ms/step - loss: 1.5480 - accuracy: 0.5112 - val_loss: 1.4823 - val_accuracy: 0.5377
Epoch 7/10
1062/1062 [==============================] - 329s 310ms/step - loss: 1.4782 - accuracy: 0.5367 - val_

In [41]:
model.evaluate(X_test, y_test.values, batch_size=128)

133/133 [==============================] - 22s 162ms/step - loss: 1.6185 - accuracy: 0.4832


[1.6185097694396973, 0.4832195043563843]